# 패키지 다운로드 & 버전 확인

In [1]:
!nvcc -V
!echo
!gcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0

gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Mon Dec 13 12:41:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+=========

In [ ]:
!git clone https://github.com/SwinTransformer/Swin-Transformer-Object-Detection.git
!pip install -r /content/Swin-Transformer-Object-Detection/requirements.txt
!pip install openmim
!mim install mmdet
!pip install tqdm
! python -m pip -q install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
%cd ..

# 데이터 다운로드 & 오류데이터 수정

In [ ]:
!rm -rf /content/data/valid

In [ ]:
import os
os.makedirs('/content/data')

In [ ]:
!unzip '/content/drive/MyDrive/input/valid.zip' -d /content/data/valid/
# !unzip '/content/drive/MyDrive/input/train.zip' -d /content/data/train

In [ ]:
!unzip '/content/drive/MyDrive/input/ocr_ancient.zip' -d /content/data

In [5]:
# 목판본_043_언해류_대학언해_01
# 목판본_043_언해류_대학언해1_01
# 필사본_104_문학류_대명영렬전2_01
# 필사본_104_문학류_대병영렬전2_01
# 필사본_105_문학류_대명영렬전3_01
# 필사본_105_문학류_대병영렬전3_01
import os
from glob import glob
json_list = glob(os.path.join('/content/data/이미지데이터','*')) # derectory
for i in json_list:
    tmp_list = glob(os.path.join(i,'*'))
    for j in tmp_list: # image파일

        if j.split('/')[-1][:-8] != i.split('/')[-1]:
            dir_name = i.split('/')[-1]
            img_number = j.split('/')[-1][-8:]
            os.rename(j, '/content/data/이미지데이터/'+dir_name+'/'+dir_name+img_number)

In [ ]:
import pycocotools.mask as mask_util
from detectron2.structures import polygons_to_bitmask
def polygon_to_rle(polygon:list, shape=(520, 704)):
    mask = polygons_to_bitmask([np.array(polygon) + 0.25], shape[0], shape[1])
    rle = mask_util.encode(np.asfortranarray(mask))
    return rle

# 데이터 전처리

In [28]:
import numpy as np
import pandas as pd
import cv2
import os
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import random

from collections import defaultdict, Counter
import json
from glob import glob


In [35]:
!rm -rf /content/data/train

In [7]:
def decide_min_max(coordinates : dict, h, w):
    min_y, max_y, min_x, max_x = 1e9, 0, 1e9, 0

    for i in coordinates:
        bbox = i['bbox']
        min_y, max_y, min_x, max_x = min(min_y, bbox[1]), max(max_y,bbox[1]+bbox[3]),\
                                    min(min_x, bbox[0]), max(max_x,bbox[0]+bbox[2])
    mini_y, maxi_y, mini_x, maxi_x = max(0,min_y-32), min(h,max_y+32), max(0,min_x-32), min(w, max_x+32)
    return mini_y, maxi_y, mini_x, maxi_x


def make_segmentation(x,y,w,h,image):
    box = image[y:y+h,x:x+w,:]
    rgb_distance = np.abs(box[:,:,0] - box[:,:,1]) + np.abs(box[:,:,1] - box[:,:,2]) + np.abs(box[:,:,2] - box[:,:,0])
    rgb_mask = np.zeros((h,w))
    for m in np.argwhere(rgb_distance < 0.7):
        rgb_mask[m[0], m[1]] = 1
    
    box_gray = np.mean(box, axis=2)
    mean_box = np.mean(box.flatten())
    under_mean = np.argwhere(box_gray <= mean_box*0.9)
    zero_arg = np.argwhere(box_gray <= np.min(box_gray.flatten())+0.01)
    tmp_mask = np.zeros((h,w))
    for m in under_mean:
        tmp_mask[m[0],m[1]] = 1
    for m in zero_arg:
        tmp_mask[m[0],m[1]] = 1
    total_mask = rgb_mask * tmp_mask
    seg = np.flip(np.argwhere(total_mask==1),axis=1) + [x,y]
    return seg

In [37]:
def make_dataset(mode='train', seg_option=False, just_image=False):

    os.makedirs(f'/content/data/{mode}', exist_ok=True)
    dir_list = glob('/content/data/라벨링데이터/*')

    count = 0

    image_idx = 0
    annotation_idx = 0

    images = [] # images.append({'id':image_idx, 'file_name': jf_name+'/'+jf_name+file_name[-4:]+'.png','width':d['Image_Width'], 'height':d['Image_Height']})
    annotations = [] # annotations.append({'id':annotations_idx, 'image_id':image_idx, 'category_id':0,\
                    # 'bbox':(x,y,w,h), 'area':w*h, 'ignore':0,'iscrowd':0,'segmentation':[]})
    for idx, i in tqdm(enumerate(dir_list)): # directory
        print(idx, i)
        if idx == 100:
            break
        dir_name = i.split('/')[-1]
        json_list = glob(os.path.join(i, '*')) # json파일
        if mode == 'train': ################################################# train, valid 비율설정
            iter = json_list[:int(len(json_list)*0.1)]
        else:
            iter = json_list[int(len(json_list)*0.9):]
        for j in iter:
            with open(j, 'r', encoding='utf-8-sig') as jfile:
                d = json.load(jfile)
                image_name = d['Image_filename']
                image_num = image_name[-4:]
                if d['Image_Char_no'] > 400:
                    continue
                
                image = plt.imread(i.replace('라벨링데이터','이미지데이터')+'/'+dir_name+image_num+'.png') # h, w, c

    #             image = np.zeros((int(d['Image_Height']), int(d['Image_Width']), 3), dtype=int)
                h,w,c = image.shape

                mini_y, maxi_y, mini_x, maxi_x = decide_min_max(d['Text_Coord'], h,w)
                
                ### ----------------- 글 주변만 자른 이미지 ----------------- ###
                result = image[mini_y:maxi_y, mini_x:maxi_x,:]
                h_, w_, c_ = result.shape
                
                ### ----------------- 가운데를 자르기 전 빈 공간이 있는지 체크 ----------------- ###
                width_check = np.array([0]*w_)

                for k in d['Text_Coord']:
                    bbox = k['bbox']
                    annotate = k['annotate']
                    width_check[bbox[0]-mini_x:bbox[0]-mini_x+bbox[2]] += 1
                
                ### ----------------- 빈 공간이 있으면 이미지 분할 ----------------- ###
                cutting = None
                start = None # 빈공간의 처음지점
                end = None # 빈공간의 마지막 지점 --> 치우치지 않게 자르기 위함
                flag = False
                if h_/w_ < 1.0:
                    for n in range(10, -25,-1):
                        if width_check[w_//2 + n] == 0 and flag == False:
                            start = w_//2 + n
                            end = w_//2 + n
                            flag = True
                        elif width_check[w_//2 + n] == 0 and flag == True:
                            end = w_//2 + n
                        elif width_check[w_//2 + n] != 0 and flag == True:
                            end = w_//2 + n
                            break
                        else:
                            continue
                    if start is None or end is None: # 가로가 긴데 못자르면 패스
                        continue
                        
                    cutting = (start+end)//2
                    
                    left_coord = []
                    right_coord = []
                    
                    for k in d['Text_Coord']:
                        bbox = k['bbox']
                        bbox_ = [bbox[0]-mini_x,bbox[1]-mini_y,bbox[2],bbox[3]] # 원래 bbox에서 여백제거한만큼 이동
                        if bbox_[0] > cutting:
                            bbox_[0] -= cutting
                            right_coord.append(bbox_)
                        else:
                            left_coord.append(bbox_)
                    
                    image_left = result[:,:cutting,:]
                    image_right = result[:,cutting:,:]
                    
                    h_left, w_left, _ = image_left.shape
                    h_right, w_right, _ = image_right.shape
                    if 2 > h_left / w_left >= 1: # 자른 후 세로가 길어야 저장
                        plt.imsave(f'/content/data/{mode}/{image_idx:05d}.png', image_left)
                        if just_image == True:
                            image_idx += 1
                            continue
                        images.append({'id':image_idx, 'file_name': f'{image_idx:05d}.png',\
                                    'width':w_left, 'height':h_left})
                        for ann in left_coord:
                            if 0 > ann[0] or ann[0]+ann[2] > w_left or 0 > ann[1] or ann[1]+ann[3] > h_left:
                                continue
                            seg = None
                            if seg_option:
                                x,y,w,h = ann[:4]
                                seg = make_segmentation(x,y,w,h,image_left)
                                annotations.append({'id':annotation_idx, 'image_id':image_idx, 'category_id':0,\
                                        'bbox':(ann[0],ann[1],ann[2],ann[3]), 'area':ann[2]*ann[3], 'ignore':0,'iscrowd':0,'segmentation':[seg.flatten().tolist()]})
                            else:
                                annotations.append({'id':annotation_idx, 'image_id':image_idx, 'category_id':0,\
                                        'bbox':(ann[0],ann[1],ann[2],ann[3]), 'area':ann[2]*ann[3], 'ignore':0,'iscrowd':0,'segmentation':[]})
                            annotation_idx += 1
                        image_idx += 1
                    if 2 > h_right / w_right >= 1:
                        plt.imsave(f'/content/data/{mode}/{image_idx:05d}.png', image_right)
                        if just_image == True:
                            image_idx += 1
                            continue
                        images.append({'id':image_idx, 'file_name': f'{image_idx:05d}.png',\
                                    'width':w_right, 'height':h_right})
                        for ann in right_coord:
                            if 0 > ann[0] or ann[0]+ann[2] > w_right or 0 > ann[1] or ann[1]+ann[3] > h_right:
                                continue
                            seg = None
                            if seg_option:
                                x,y,w,h = ann[:4]
                                seg = make_segmentation(x,y,w,h,image_right)
                                annotations.append({'id':annotation_idx, 'image_id':image_idx, 'category_id':0,\
                                        'bbox':(ann[0],ann[1],ann[2],ann[3]), 'area':ann[2]*ann[3], 'ignore':0,'iscrowd':0,'segmentation':[seg.flatten().tolist()]})
                            else:
                                annotations.append({'id':annotation_idx, 'image_id':image_idx, 'category_id':0,\
                                        'bbox':(ann[0],ann[1],ann[2],ann[3]), 'area':ann[2]*ann[3], 'ignore':0,'iscrowd':0,'segmentation':[]})
                            annotation_idx += 1
                        image_idx += 1
                           
                else: # 세로가 길면
                    if h_/w_ > 2 or d['Image_Char_no'] > 200:
                        continue
                    plt.imsave(f'/content/data/{mode}/{image_idx:05d}.png', result)
                    if just_image == True:
                        image_idx += 1
                        continue
                    images.append({'id':image_idx, 'file_name': f'{image_idx:05d}.png',\
                                'width':w_, 'height':h_})
                    for ann in d['Text_Coord']:
                        x,y,w,h = ann['bbox'][:4]
                        if 0 > x-mini_x or x+w-mini_x > w_ or 0 > y-mini_y or y+h-mini_y > h_:
                            continue
                        seg = None
                        if seg_option:
                            seg = make_segmentation(x-mini_x,y-mini_y,w,h,result)
                            annotations.append({'id':annotation_idx, 'image_id':image_idx, 'category_id':0,\
                                    'bbox':(x-mini_x,y-mini_y,w,h), 'area':w*h, 'ignore':0,'iscrowd':0,'segmentation':[seg.flatten().tolist()]})
                        else:
                            annotations.append({'id':annotation_idx, 'image_id':image_idx, 'category_id':0,\
                                    'bbox':(x-mini_x,y-mini_y,w,h), 'area':w*h, 'ignore':0,'iscrowd':0,'segmentation':[]})
                        annotation_idx += 1
                    image_idx += 1
    return images, annotations

In [33]:
def save_json(images, annotations, mode='train'):
    coco_dict = dict(
        info= dict(
            year=2021, 
            version="1.0", 
            description="OCR", 
            contributor="", 
            url=None, 
            date_created=None
        ), # year, version, description, contributor, url, date_created
        licenses=dict(
            id=0,
            name='CC BY 4.0',
            url=''
        ), # id, name, url
        images=list(), # id, file_name, height, width
        annotations=list(), # id, image_id, category_id, bbox, area, iscrowd
        categories = [
            dict(id=0,
                name='word')
        ] # id, name, supercategory
    )
    coco_dict['images'] = images
    coco_dict['annotations'] = annotations
    coco_dict['categories'] = [{'id':0, 'name':'word','super_category':None}]
    with open(f'/content/data/{mode}.json', 'w', encoding='utf-8') as jfile:
        json.dump(coco_dict, jfile)
    # with open(f'/content/drive/MyDrive/detection/{mode}.json', 'w', encoding='utf-8') as jfile:
    #     json.dump(coco_dict, jfile)

In [ ]:
images, annotations = make_dataset('train', seg_option=True, just_image=False)
save_json(images, annotations,'train')
del images, annotations
images, annotations = make_dataset('valid', seg_option=True, just_image=False)
save_json(images, annotations,'valid')

## 데이터 정상인지 확인

In [46]:
# mode = 'valid'
# with open(f'/content/data/{mode}.json','r', encoding='utf-8') as f:
#     d = json.load(f)
mode = 'valid'
# with open(f'/content/drive/MyDrive/detection/{mode}.json','r', encoding='utf-8') as f:
#     d = json.load(f)
with open(f'/content/data/{mode}.json','r',encoding='utf-8') as f:
    d = json.load(f)
for k in range(20):
    image_sample = d['images'][k]

    ann_sample = []
    for i in d['annotations']:
        if i['image_id'] == image_sample['id']:
            ann_sample.append(i)
    print(len(ann_sample))
    f_name = image_sample['file_name']
    img = plt.imread(f'/content/data/{mode}/{f_name}')
    plt.imshow(img)
    plt.show()
    fig, ax = plt.subplots(figsize=(12,12))
    
    ## segmentation
    tmp = np.zeros((image_sample['height'], image_sample['width']))
    for j in ann_sample:
        for m in range(0, len(j['segmentation'][0]), 2):
            tmp[j['segmentation'][0][m+1], j['segmentation'][0][m]] = 1
    ax.imshow(tmp)
    plt.show()

    ## bounding box
    # ax.imshow(img)
    # for j in ann_sample:
    #     x,y,w,h = j['bbox']
    #     rect = patches.Rectangle((x,y),w,h, linewidth=3, edgecolor='r', facecolor='none')
    #     ax.add_patch(rect)
    # plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [12]:
cnt = 0
for i in range(len(d['annotations'])):
    if len(d['annotations'][i]['segmentation'][0]) <= 4:
        print(d['annotations'][i])
        image_id = d['annotations'][i]['image_id']
        img = plt.imread(f'/content/data/valid/{image_id:05d}.png')
        # fig, ax = plt.subplots(figsize=(12,12))
        
        ax.imshow(img)
        for j in d['annotations']:
            x,y,w,h = j['bbox']
            
            if j['image_id'] == image_id and len(j['segmentation'][0])<=4:
                print(j['segmentation'][0])
                # rect = patches.Rectangle((x,y),w,h, linewidth=3, edgecolor='r', facecolor='none')
                # ax.add_patch(rect)
                cnt += 1
                bbox = img[y:y+h, x:x+w, :]
                plt.imshow(bbox)
                plt.show()
        # plt.show()

cnt # 348개의 글자가 거의 사라짐 - mean이 문제일수도 -> 1.0 mean으로 바꾸니 3개로 줄엇다 -> mean 값으로 설정하면 검은색이 큰 경우에 
# 4개 이하 1223개 -> 맥스값의 절반은 어떨까?

0

In [ ]:
rm -rf /content/data/valid

In [ ]:
!zip '/content/drive/MyDrive/input/train.zip' /content/data/train/*

In [ ]:
!zip '/content/drive/MyDrive/input/valid.zip' /content/data/valid/*

# MMDetection 학습

In [39]:
%cd Swin-Transformer-Object-Detection/

[Errno 2] No such file or directory: 'Swin-Transformer-Object-Detection/'
/content/Swin-Transformer-Object-Detection


In [40]:
import sys
from mmcv import Config
from mmcv.runner import load_checkpoint
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector, set_random_seed, init_detector
from mmdet.datasets import build_dataloader, build_dataset, replace_ImageToTensor
from mmdet.utils import collect_env, get_root_logger

import torch
import time
import json

from pprint import pprint
import os

In [41]:
classes = (
    "word",
)
CUR_PATH = os.getcwd()
CFG_PATH = "/content/drive/Othercomputers/내 컴퓨터/workspace/ocr_ancient_korean/detection/configs/mask_rcnn/mask_rcnn_r50_fpn_1x_coco.py"
# CFG_PATH = "/content/drive/Othercomputers/내 컴퓨터/workspace/ocr_ancient_korean/detection/configs/swin/cascade_mask_rcnn_swin_base_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco.py"
PREFIX = "/content/data/image"
WORK_DIR = "/content/drive/MyDrive/Colab Notebooks/Computer_Vision/work_dir"
# CHK_PATH = "/content/drive/MyDrive/Colab Notebooks/lesion_obd/cascade_mask_rcnn_swin_small_patch4_window7.pth"
# config file 들고오기
cfg = Config.fromfile(CFG_PATH)

In [ ]:
for i in cfg:
    print(i, '*'*40)
    print(cfg[i])

In [42]:

cfg.data.train.classes = classes
cfg.data.train.img_prefix = "/content/data/train"
cfg.data.train.ann_file = '/content/data/train.json'

cfg.data.val.classes = classes
cfg.data.val.img_prefix = "/content/data/valid"
cfg.data.val.ann_file = '/content/data/valid.json'

cfg.data.samples_per_gpu = 2
cfg.data.workers_per_gpu = 4

cfg.seed = 42
cfg.gpu_ids = [0]

distributed = False

cfg.work_dir = WORK_DIR
cfg.runner.max_epochs = 20
cfg.rtotal_epochs = 20
cfg.optimizer = dict(type='Adam', lr=0.0001, weight_decay=0.0001)

cfg.lr_config = dict(
    policy='CosineAnnealing', # The policy of scheduler, also support CosineAnnealing, Cyclic, etc. Refer to details of supported LrUpdater from https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/lr_updater.py#L9.
    by_epoch=False,
    warmup='linear', # The warmup policy, also support `exp` and `constant`.
    warmup_iters=1000, # The number of iterations for warmup
    warmup_ratio=0.001, # The ratio of the starting learning rate used for warmup
    min_lr=1e-07)

cfg.log_config.interval = 600
cfg.checkpoint_config.interval = 1
cfg.log_config = {'hooks': [{'type': 'TextLoggerHook'}], 'interval': 600}
# cfg.model.pretrained = None

# cfg.load_from = '/content/drive/Othercomputers/내 컴퓨터/workspace/ocr_ancient_korean/detection/pth/cascade_mask_rcnn_swin_base_patch4_window7.pth'

model = build_detector(cfg.model)

datasets = [build_dataset(cfg.data.train)]
model.CLASSES = datasets[0].CLASSES

2021-12-13 14:26:24,953 - mmdet - INFO - load model from: torchvision://resnet50
2021-12-13 14:26:24,955 - mmdet - INFO - load checkpoint from torchvision path: torchvision://resnet50
2021-12-13 14:26:25,067 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



loading annotations into memory...
Done (t=2.53s)
creating index...
index created!


In [ ]:
import gc
gc.collect()

88

In [43]:
train_detector(model, datasets[0], cfg, distributed=distributed, validate=True)

loading annotations into memory...


2021-12-13 14:26:33,610 - mmdet - INFO - Start running, host: root@a76640b634de, work_dir: /content/drive/MyDrive/Colab Notebooks/Computer_Vision/work_dir
2021-12-13 14:26:33,617 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) EvalHook                           
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook                      
(NORMAL

Done (t=3.00s)
creating index...
index created!


IndexError: ignored